In [ ]:
import csv
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def load_data(filename):
    return np.loadtxt(open(filename), delimiter=',', skiprows=1)

In [ ]:
# Load training data
data = load_data('train_2008.csv')
X = data[:, 1:-1]
y = data[:, -1]

In [ ]:
N = len(y) #64667
D = len(X[0]) #381

# Split to train / test (validation) data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_sample, y_sample = X_train[:1000], y_train[:1000]

In [ ]:
# Load real test data
test_data = load_data('test_2008.csv')
X_test_real = test_data[:, 1:]

In [ ]:
#Gradient Boosting 
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
y_score_gbr = gbr.predict(X_test)


In [ ]:
gbr.coef_

In [ ]:
# Logistic Regression
lgr = LogisticRegression(C = 0.1, penalty = 'l1')
lgr.fit(X_train, y_train)
pred_prob = lgr.predict_proba(X_test)
y_scores_lgr = pred_prob[:,1]

In [ ]:
# Find features with the most contribution

weights = lgr.coef_.flatten()
top_idxs = np.argsort(abs(weights))[-10:]
features = np.loadtxt(open('test_2008.csv'), delimiter=',', dtype='str')[0, 1:]


In [ ]:
print(features[top_idxs])
[weights[i] for i in top_idxs]

In [ ]:
weights = list(lgr.coef_[0])

for w in weights:

In [ ]:
# Random Forest Regression
regr = RandomForestRegressor(n_estimators = 100, max_features = 26)
regr.fit(X_train, y_train)
y_score_regr = regr.predict(X_test)

In [ ]:
# average of the three
y_test_scores = 0.4*y_score_gbr + 0.3*y_scores_lgr + 0.3*y_score_regr


y_test_scores = y_test_scores.clip(min=0)
roc_auc_score(y_test, y_test_scores)

In [ ]:
# Train on the entire training data
gbr = GradientBoostingRegressor()
gbr.fit(X, y)

# Load real test data
test_data = load_data('test_2008.csv')
X_test_real = test_data[:, 1:]

# Output results on the actual test data
y_test_scores = gbr.predict(X_test_real)

In [ ]:
y_test_scores = y_test_scores.clip(min=0)

In [ ]:
with open ('sub1.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerow(['id', 'target'])
    for i in range(0, 16000):
        writer.writerow([i, y_test_scores[i]])

In [ ]:
# Gradient Boosting Regressor with Grid Search
gbr = GradientBoostingRegressor()

param_grid={'n_estimators':[100,200,500], 
            'learning_rate': [0.1,0.05,0.02],
            'max_depth':[3,4], 
            'min_samples_leaf':[1,2,3], 
            'max_features':[None,1.0] } 

gbr = GridSearchCV(estimator=gbr, cv=5, param_grid=param_grid, 
    n_jobs=4)

gbr.fit(X_sample, y_sample)
y_score_gbr = gbr.predict(X_test)

print(roc_auc_score(y_test, y_score_gbr))

print(gbr.best_params_)